# Notebook 02 — Camada Analítica (SQLite → BI)

**Objetivo:** ler os dados **certificados** no SQLite e gerar tabelas prontas para dashboard:

- `fato_vendas_analitica` (nível de venda / linha)
- `vendas_mensais` (agregado por mês)
- `meta_vs_realizado` (meta x realizado por mês)

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

In [2]:
DB = "../database/dados_tratados.db"

with sqlite3.connect(DB) as conn:
    vendas = pd.read_sql("SELECT * FROM vendas_certificadas", conn)
    produtos = pd.read_sql("SELECT * FROM produtos_certificados", conn)
    metas = pd.read_sql("SELECT * FROM metas_certificadas", conn)

print("Linhas vendas:", len(vendas))
print("Linhas produtos:", len(produtos))
print("Linhas metas:", len(metas))

display(vendas.head())

Linhas vendas: 200
Linhas produtos: 10
Linhas metas: 6


,data_venda,id_produto,id_cliente,canal_venda,quantidade,preco_original_registrado,preco_unitario_oficial
0,2024-06-22 00:00:00,3,27,Marketplace,4,99.9,3999.9
1,2024-05-19 00:00:00,4,43,WhatsApp,3,79.9,149.9
2,2024-03-16 00:00:00,7,12,WhatsApp,1,79.9,699.9
3,2024-02-28 00:00:00,10,20,WhatsApp,5,249.9,349.9
4,2024-04-05 00:00:00,4,48,WhatsApp,2,199.9,149.9


In [3]:
vendas["data_venda"] = pd.to_datetime(vendas["data_venda"], errors="coerce")
vendas["quantidade"] = pd.to_numeric(vendas["quantidade"], errors="coerce")
vendas["preco_unitario_oficial"] = pd.to_numeric(vendas["preco_unitario_oficial"], errors="coerce")

if "custo_unitario" in produtos.columns:
    produtos["custo_unitario"] = pd.to_numeric(produtos["custo_unitario"], errors="coerce")

if "meta_faturamento" in metas.columns:
    metas["meta_faturamento"] = pd.to_numeric(metas["meta_faturamento"], errors="coerce")


## 3) Criar a Fato Analítica

- `ano_mes` (YYYY-MM) para facilitar gráficos
- `receita` = quantidade × preço oficial
- `custo_total`, `lucro`, `margem`


In [4]:
# Join com produtos (para trazer custo_unitariO)
cols_prod = ["id_produto"]
if "custo_unitario" in produtos.columns:
    cols_prod.append("custo_unitario")

fato = vendas.merge(produtos[cols_prod], on="id_produto", how="left")

# ano_mes
fato["ano_mes"] = fato["data_venda"].dt.to_period("M").astype(str)

# receita
fato["receita"] = fato["quantidade"] * fato["preco_unitario_oficial"]

# custo/lucro/margem

fato["custo_total"] = fato["quantidade"] * fato["custo_unitario"]
fato["lucro"] = fato["receita"] - fato["custo_total"]
fato["margem"] = (fato["lucro"] / fato["receita"]).where(fato["receita"] != 0)

# Colunas finais (simples e úteis)
cols_fato = [
    "data_venda","ano_mes","id_produto","id_cliente","canal_venda","quantidade",
    "preco_unitario_oficial","receita","custo_total","lucro","margem"
]
cols_fato = [c for c in cols_fato if c in fato.columns]
fato = fato[cols_fato].copy()

display(fato.head())

,data_venda,ano_mes,id_produto,id_cliente,canal_venda,quantidade,preco_unitario_oficial,receita,custo_total,lucro,margem
0,2024-06-22,2024-06,3,27,Marketplace,4,3999.9,15999.6,11200,4799.6,0.299982
1,2024-05-19,2024-05,4,43,WhatsApp,3,149.9,449.7,240,209.7,0.466311
2,2024-03-16,2024-03,7,12,WhatsApp,1,699.9,699.9,300,399.9,0.571367
3,2024-02-28,2024-02,10,20,WhatsApp,5,349.9,1749.5,1000,749.5,0.428408
4,2024-04-05,2024-04,4,48,WhatsApp,2,149.9,299.8,160,139.8,0.466311


## 4) Agregação mensal
tabela resumida por mês (para o dashboard).

In [5]:
vendas_mensais = (
    fato.groupby("ano_mes", as_index=False)
        .agg(
            receita=("receita", "sum"),
            quantidade=("quantidade", "sum"),
            vendas=("ano_mes", "count")
        )
)

if "lucro" in fato.columns and fato["lucro"].notna().any():
    lucro_mensal = fato.groupby("ano_mes", as_index=False).agg(lucro=("lucro", "sum"))
    vendas_mensais = vendas_mensais.merge(lucro_mensal, on="ano_mes", how="left")
    vendas_mensais["margem"] = (vendas_mensais["lucro"] / vendas_mensais["receita"]).where(vendas_mensais["receita"] != 0)
else:
    vendas_mensais["lucro"] = pd.NA
    vendas_mensais["margem"] = pd.NA

display(vendas_mensais)


,ano_mes,receita,quantidade,vendas,lucro,margem
0,2024-01,68111.7,83,31,26711.7,0.392175
1,2024-02,41143.1,69,21,16833.1,0.409135
2,2024-03,57032.1,79,29,23342.1,0.409280
3,2024-04,102386.8,132,44,38956.8,0.380487
4,2024-05,76559.9,101,35,32499.9,0.424503
5,2024-06,127007.2,128,40,47297.2,0.372398


## 5) Meta vs Realizado

Junta `metas_certificadas` com `vendas_mensais` para comparar meta e realizado.


In [6]:
# 5) Meta vs Realizado

if "ano_mes" not in metas.columns:
    if "mes" in metas.columns:
        metas = metas.rename(columns={"mes": "ano_mes"})
    else:
        raise ValueError("A tabela metas_certificadas precisa ter uma coluna 'ano_mes' (ou 'mes').")

meta_vs_realizado = metas.merge(
    vendas_mensais[["ano_mes","receita","lucro","margem"]],
    on="ano_mes",
    how="left"
)

if "meta_faturamento" in meta_vs_realizado.columns:
    meta_vs_realizado["atingimento_meta"] = (meta_vs_realizado["receita"] / meta_vs_realizado["meta_faturamento"]).where(meta_vs_realizado["meta_faturamento"] != 0)
else:
    meta_vs_realizado["atingimento_meta"] = pd.NA
    print("⚠️ Não encontrei 'meta_faturamento' em metas_certificadas. Vou salvar mesmo assim, mas sem atingimento.")

display(meta_vs_realizado)

,ano_mes,meta_faturamento,receita,lucro,margem,atingimento_meta
0,2024-01,59725,68111.7,26711.7,0.392175,1.140422
1,2024-02,55353,41143.1,16833.1,0.409135,0.743286
2,2024-03,38115,57032.1,23342.1,0.409280,1.496316
3,2024-04,35387,102386.8,38956.8,0.380487,2.893345
4,2024-05,53919,76559.9,32499.9,0.424503,1.419906
5,2024-06,59945,127007.2,47297.2,0.372398,2.118729


## 6) Salvar tabelas analíticas no SQLite

In [7]:
# 6) Persistência no SQLite
with sqlite3.connect(DB) as conn:
    fato.to_sql("fato_vendas_analitica", conn, if_exists="replace", index=False)
    vendas_mensais.to_sql("vendas_mensais", conn, if_exists="replace", index=False)
    meta_vs_realizado.to_sql("meta_vs_realizado", conn, if_exists="replace", index=False)

In [8]:
with sqlite3.connect(DB) as conn:
    chk = pd.read_sql("""
        SELECT
          (SELECT COUNT(*) FROM fato_vendas_analitica) AS linhas_fato,
          (SELECT COUNT(*) FROM vendas_mensais) AS linhas_mensais,
          (SELECT COUNT(*) FROM meta_vs_realizado) AS linhas_meta
    """, conn)

display(chk)

,linhas_fato,linhas_mensais,linhas_meta
0,200,6,6
